In [ ]:
!pip install requests
!pip install xmltodict
!pip install pandas
!pip install nltk
!pip install jsonlines
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
import requests
import xmltodict
import json
import csv
import re
from datetime import datetime, timedelta
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import google.generativeai as genai
from google.colab import userdata
import time

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Pré-Processamento

In [ ]:
def limpar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        texto = texto.translate(str.maketrans('', '', string.punctuation))
    return texto

def remover_stopwords(texto):
    if isinstance(texto, str):
        stop_words = set(stopwords.words('portuguese'))
        tokens = word_tokenize(texto)
        tokens_filtrados = [token for token in tokens if token not in stop_words]
        return ' '.join(tokens_filtrados)
    return texto

def lematizar_texto(texto):
    if isinstance(texto, str):
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(texto)
        lematizado_tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
        return ' '.join(lematizado_tokens)
    return texto

def pre_processar_texto(texto):
    texto = limpar_texto(texto)
    texto = remover_stopwords(texto)
    texto = lematizar_texto(texto)
    return texto

Busca dos Dados

In [ ]:
def buscar_dados_do_mes(ano, mes):
    data_str = f"{ano}{mes:02}01"
    url = f"https://legis.senado.leg.br/dadosabertos/plenario/resultado/mes/{data_str}"
    resposta = requests.get(url)

    if resposta.status_code == 200:
        data_dict = xmltodict.parse(resposta.content)
        if 'Sessoes' in data_dict.get('ResultadoPlenarioEplen', {}):
            sessoes = data_dict['ResultadoPlenarioEplen']['Sessoes']['Sessao']
            return sessoes
        else:
            print(f"Nenhuma sessão encontrada para o mês {mes}/{ano}")
            return []
    else:
        print(f"Erro ao acessar o endpoint para {data_str}: {resposta.status_code}")
        return []

Extrair e Processar os Dados

In [ ]:
def extrair_e_processar_dados(data_inicio, data_fim):
    todos_discursos = []
    data_atual = data_inicio

    while data_atual <= data_fim:
        discursos = buscar_dados_do_mes(data_atual.year, data_atual.month)
        for sessao in discursos:
            data_sessao = datetime.strptime(sessao['dataSessao'], "%d/%m/%Y").date()
            itens = sessao.get('Itens', {}).get('Item', [])

            if isinstance(itens, list):
                for item in itens:
                    dados_discurso = {
                        'numero_decreto': item.get('numeroMateria', "nao possui"),
                        'data_sessao': data_sessao.strftime("%Y-%m-%d"),
                        'proponente': pre_processar_texto(item.get('autorMateria', "nao possui")),
                        'senador_proponente': pre_processar_texto(item.get('autorMateria', "nao possui")),
                        'relator': pre_processar_texto(item.get('parecer', "nao possui").split('Relator: ')[1].split(',')[0] if 'Relator: ' in item.get('parecer', "") else "nao possui"),
                        'tema_resumido': pre_processar_texto(item.get('ementaPapeleta', "nao possui")),
                        'categoria': ''
                    }
                    todos_discursos.append(dados_discurso)

        proximo_mes = data_atual.replace(day=28) + timedelta(days=4)
        data_atual = proximo_mes.replace(day=1)

    return todos_discursos

Salvar os Dados em JSONL e CSV

In [ ]:
def salvar_dados(dados, nome_arquivo_base):
    with open(f'{nome_arquivo_base}.jsonl', 'w', encoding='utf-8') as arquivo_jsonl, \
         open(f'{nome_arquivo_base}.csv', 'w', newline='', encoding='utf-8') as arquivo_csv:

        writer = csv.DictWriter(arquivo_csv, fieldnames=dados[0].keys())
        writer.writeheader()

        for discurso in dados:
            json.dump(discurso, arquivo_jsonl, ensure_ascii=False)
            arquivo_jsonl.write('\n')
            writer.writerow(discurso)


Interação com o Gemini Pro

In [ ]:
def analisar_dados_com_IA(df, model, page_size=100):
    chat = model.start_chat(history=[])

    # Calcula o número de lotes
    num_batches = -(-len(df) // page_size)

    prompt = input("Qual o comando? ")

    while prompt != "fim":
        for batch_num in range(num_batches):

            start_idx = batch_num * page_size
            end_idx = min((batch_num + 1) * page_size, len(df))

            relevant_data = df[start_idx:end_idx]

            context = f"Dados relevantes:\n\n{relevant_data.to_markdown(index=False, numalign='left', stralign='left')}\n\nPergunta: {prompt}"

            try:
                resposta = chat.send_message(context)
                print(resposta.text)
            except Exception as e:
                if "429" in str(e):
                    print("Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...")
                    time.sleep(60)
                    continue
                else:
                    raise e


In [ ]:
if __name__ == "__main__":
    data_inicio = datetime(2023, 5, 1)
    data_fim = datetime(2024, 5, 1)

    # Extração e processamento dos dados
    todos_discursos = extrair_e_processar_dados(data_inicio, data_fim)

    # Salvamento dos dados
    salvar_dados(todos_discursos, 'dados_agregados')

    # Carregamento dos dados pré-processados para análise com IA
    df_pre_processado = pd.read_csv('dados_agregados.csv')

    # Configuração do modelo Gemini Pro (substitua pela sua API KEY)
    API_KEY = userdata.get('API_KEY')
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-1.5-pro')

    # Análise dos dados com IA
    analisar_dados_com_IA(df_pre_processado, model)

Nenhuma sessão encontrada para o mês 1/2024
Qual o comando? preencha a coluna categoria de acordo com o contexto presente em tema_resumido, exemplo, saude, educacao, infraestrutura,somente uma categoria por linha, quando o tema se mencionar desarquivamento considere a categoria como judicial em csv
```csv
numero_decreto,data_sessao,proponente,senador_proponente,relator,tema_resumido,categoria
338,2023-05-02,senador romário,senador romário,senador paulo paim,altera lei nº 9250 26 dezembro 1995 tornar dedutíveis imposto sobre renda pessoa física doações feitas fundos controlados conselhos direitos pessoas deficiência,social
1150,2023-05-02,presidência república,presidência república,nao possui,altera lei nº 12651 25 maio 2012 dispõe sobre proteção vegetação nativa,meio ambiente
1151,2023-05-02,presidência república,presidência república,deputado zé vitor,altera lei nº 11284 2006 dispõe sobre gestão florestas públicas produção sustentável lei nº 11516 2007 dispõe sobre criação instituto c

Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...


Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...


Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...


Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...


Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...


Muitas solicitações. Aguardando 60 segundos antes de tentar novamente...


KeyboardInterrupt: 

Início da Execução

In [ ]:
if __name__ == "__main__":
    data_inicio = datetime(2023, 5, 1)
    data_fim = datetime(2024, 5, 1)

    # Extração e processamento dos dados
    todos_discursos = extrair_e_processar_dados(data_inicio, data_fim)

    # Salvamento dos dados
    salvar_dados(todos_discursos, 'dados_agregados')

    # Carregamento dos dados pré-processados para análise com IA
    df_pre_processado = pd.read_csv('dados_agregados.csv')

    # Configuração do modelo Gemini Pro (substitua pela sua API KEY)
    API_KEY = userdata.get('API_KEY')
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-1.5-pro')

    # Análise dos dados com IA
    analisar_dados_com_IA(df_pre_processado, model)

Nenhuma sessão encontrada para o mês 1/2024
Qual o comando? preencha a coluna categoria de acordo com o contexto presente em tema_resumido, exemplo, saude, educacao, infraestrutura
## Dados relevantes com coluna "categoria" preenchida:

| numero_decreto | data_sessao | proponente | senador_proponente | relator | tema_resumido | categoria |
|---|---|---|---|---|---|---| 
| 338 | 2023-05-02 | senador romário | senador romário | senador paulo paim | altera lei nº 9250 26 dezembro 1995 tornar dedutíveis imposto sobre renda pessoa física doações feitas fundos controlados conselhos direitos pessoas deficiência | **Direitos Humanos e Sociais** |
| 1150 | 2023-05-02 | presidência república | presidência república | nao possui | altera lei nº 12651 25 maio 2012 dispõe sobre proteção vegetação nativa | **Meio Ambiente** |
| 1151 | 2023-05-02 | presidência república | presidência república | deputado zé vitor | altera lei nº 11284 2006 dispõe sobre gestão florestas públicas produção sustentável l